In [ ]:
!pip install transformers  datasets spacy conllu


In [ ]:
import conllu

def load_conllu(filename):
    for sentence in conllu.parse(open(filename, "rt", encoding="utf-8").read()):
        tokenized_words = [token["form"] for token in sentence]
        gold_tags = [token["upos"] for token in sentence]
        yield tokenized_words, gold_tags

corpus = list(load_conllu("/kaggle/input/sequoi-ud-fr/fr_sequoia-ud-test.conllu"))


In [ ]:
import pandas as pd


def conllu2df(corpus:list) -> pd.DataFrame:

    df = pd.DataFrame(corpus, columns=["Words", "Tags"])
    df["SentenceID"] = df.index
    df = df.explode(["Words", "Tags"], ignore_index=True)     # Use explode to flatten each column
    df = df[["SentenceID", "Words", "Tags"]]
    df.columns = ["sentenceID", "token", "tag"]
    return df

In [ ]:
df = conllu2df(corpus)
df['tag'].value_counts().plot.bar()


In [ ]:
# multiword tokens
df[df['tag'] == '_']['token'].unique()

In [ ]:
# The multiword tokens are annotated by ' _ '
df[df['sentenceID'] == 2]

In [ ]:
# NUM tags contains Spaces They should be ommited
df[df['token'].str.contains(' ')]['tag'].value_counts()

# Removing the spaces from the tokens
df['token'] = df['token'].str.replace(' ', '', regex=False)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


In [ ]:
UD_tokens = df[df['sentenceID'] == 23]['tag'].to_list()
print(UD_tokens)

In [ ]:
print("UD Tokenization:")
tokens = df[df['sentenceID'] == 23]['token'].to_list()
print(tokens)
print(len(tokens))
print()


print("mBert Tokenization:")
text= " ".join(df[df['sentenceID'] == 23]['token'].to_list())
print(tokenizer.tokenize(text))
print(len(tokenizer.tokenize(text)))


In [ ]:
sentences = df[df['tag'] == '_']['sentenceID']

print("The sentences that contains multiwords")
print(sentences.unique())


multitokens_df = df[df['sentenceID'].isin(sentences.unique())]

In [ ]:
tokens  = multitokens_df[multitokens_df['sentenceID'] == 5]['token'].to_list()
tags    = multitokens_df[multitokens_df['sentenceID'] == 5]['tag'].to_list()

In [ ]:
print(tags)

assert len(tokens) == len(tags)

In [ ]:
def update_UD_tokenization(tokens:list[str], tags:list[str]):
    new_tags = []
    for i in range(len(tags) -2):
        print(i)

        if tags[i] == '_':
            print("index of _ is ",i)
            new_tags.append(tags[i+1]+'+'+tags[i+2])


            print(new_tags)

        else:
            new_tags.append(tags[i])


        while i <len(tags) -2:
            print(tags[i])



